# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jember,-8.1660,113.7032,22.52,93,62,0.29,ID,1727628959
1,1,iqaluit,63.7506,-68.5145,7.85,65,75,4.12,CA,1727628747
2,2,ajdabiya,30.7554,20.2263,26.26,77,0,6.70,LY,1727628961
3,3,masterton,-40.9597,175.6575,3.82,89,7,0.46,NZ,1727628962
4,4,tiksi,71.6872,128.8694,-6.00,94,90,2.21,RU,1727628963


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 1.0,
    size="Humidity",
    color = "City"
)

# Display the map plot
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
#  Narrow down cities that fit the criteria
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp'] < 27) &                 # Max Temp lower than 27
    (city_data_df['Max Temp'] > 21) &                 # Max Temp higher than 21
    (city_data_df['Wind Speed'] < 4.5)                # Wind Speed less than 4.5 m/s             
]

 # Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
sample_size = min(20, len(filtered_cities_df))  # Ensure sample size does not exceed available data
sample_data = filtered_cities_df.sample(sample_size)  
display(sample_data)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
283,283,ifaki,7.7894,5.2485,24.02,85,90,0.52,NG,1727629273
38,38,talodi,10.6325,30.3797,21.53,94,77,4.33,SD,1727628999
134,134,hiratacho,35.4333,132.8167,22.98,83,75,4.12,JP,1727629103
251,251,fountain,38.6822,-104.7008,26.59,30,0,3.09,US,1727629235
257,257,sorong,-0.8833,131.2500,26.41,82,87,1.63,ID,1727629240
299,299,hamilton,39.1834,-84.5333,21.06,95,100,2.06,US,1727629054
280,280,tsiombe,-25.3000,45.4833,23.47,79,97,2.69,MG,1727629268
553,553,roha,18.4333,73.1167,24.13,95,6,0.76,IN,1727629577
458,458,harper,4.3750,-7.7169,25.93,83,100,3.44,LR,1727629469
54,54,manthani,18.6500,79.6667,26.44,84,100,1.95,IN,1727629018


### Step 3: Create a new DataFrame called `hotel_df`.

In [47]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Lat', 'Lng','Humidity','Country']].copy()  # Adjust based on available columns

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
sample_hotel_data = hotel_df.sample(20) 
display(sample_hotel_data)

,City,Lat,Lng,Humidity,Country,Hotel Name
253,pontianak,-0.0333,109.3333,100,ID,
257,sorong,-0.8833,131.2500,82,ID,
499,kununurra,-15.7667,128.7333,78,AU,
259,santo nino,6.4350,124.6975,94,PH,
119,sadjoavato,-12.6334,49.3490,86,MG,
468,wenchang,30.9040,118.4887,91,CN,
190,saint-pierre,-21.3393,55.4781,83,RE,
462,el tocuyo,9.7873,-69.7937,93,VE,
310,meulaboh,4.1363,96.1285,87,ID,
458,harper,4.3750,-7.7169,83,LR,


# Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [36]:
# Set parameters to search for a hotel
categories="accommodation.hotel"
conditions= ""
radius = 10000
filters=""
bias=""

params = {
    "categories":categories,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"][index]
    longtitude = hotel_df["Lng"][index]
        
    # Add the current city's latitude and longitude to the params dictionary
    filters=f"circle:{longtitude},{latitude},{radius}"
    bias=f"proximity:{longtitude},{latitude}"
    
    params["filter"] = filters
    params["bias"] = bias

    #print(params)

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response=requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    #print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
sample_data = filtered_cities_df.sample(20)  # Adjust the number of samples as needed
display(sample_data)

Starting hotel search
jember - nearest hotel: Hotel Asri
mounana - nearest hotel: No hotel found
viqueque - nearest hotel: Hotel Wailakurini
talodi - nearest hotel: No hotel found
touba - nearest hotel: Hôtel de l'amitié vrai
moyale - nearest hotel: Paul’s guest house
manthani - nearest hotel: No hotel found
antalaha - nearest hotel: Hôtel Florida
hawaiian paradise park - nearest hotel: No hotel found
axim - nearest hotel: Axim Beach Hotel
avarua - nearest hotel: Paradise Inn
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
mao - nearest hotel: Hevresac
khuzdar - nearest hotel: Al-Mustafa hotel
n'dalatando - nearest hotel: Hotel Terminus
kailua-kona - nearest hotel: Kona Seaside Hotel
sadjoavato - nearest hotel: No hotel found
cochin - nearest hotel: Hotel Top Form
taiping - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
hiratacho - nearest hotel: ひらたメイプルホテル
hargeysa - nearest hotel: No hotel found
haimen - nearest hotel: No hotel found
benguela 

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
42,42,touba,8.2833,-7.6833,25.89,87,71,1.72,CI,1727629003
468,468,wenchang,30.9040,118.4887,23.02,91,46,3.66,CN,1727629481
328,328,nova vicosa,-17.8919,-39.3719,25.83,70,67,2.53,BR,1727629301
253,253,pontianak,-0.0333,109.3333,23.99,100,40,1.03,ID,1727629238
375,375,bordj ghdir,35.9011,4.8981,21.16,58,1,2.79,DZ,1727629377
53,53,moyale,3.5167,39.0584,25.58,41,95,3.46,KE,1727629017
356,356,chui,-33.6971,-53.4616,22.65,63,14,3.31,UY,1727629355
323,323,lata,40.1629,-8.3327,22.48,31,7,2.76,PT,1727629318
205,205,veraval,20.9000,70.3667,26.83,87,100,3.61,IN,1727629180
207,207,toliara,-23.3500,43.6667,23.48,80,50,2.18,MG,1727629182


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng", 
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    scale= 1.0,
    color="City",
    hover_cols=["Humidity","Hotel Name","Country"]
)

# Display the map plot
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)